In [1]:
from glob import glob
import os
import joblib
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
a_file_path=glob('derivatives/A/*.set')
f_file_path=glob('derivatives/F/*.set')

In [3]:
print(len(a_file_path),len(f_file_path))

36 23


In [4]:
eeg_channels = ['F3',
 'F4',
 'F7',
 'F8',
 'T3',
 'T4',
 'T5',
 'T6',
 'Fz']

In [5]:
def read_data(file_path):
    data=mne.io.read_raw_eeglab(file_path,preload=True)
    data.set_eeg_reference()
    data = data.pick_channels(eeg_channels)#for picking frontotemporal regions
    #data.filter(l_freq=0.5,h_freq=45)
    epochs=mne.make_fixed_length_epochs(data,duration=5,overlap=1)
    array=epochs.get_data()
    return array

In [6]:
def read_data_test(file_path):
    data=mne.io.read_raw_edf(file_path,preload=True)
    data.set_eeg_reference()
    data.filter(l_freq=0.5,h_freq=45)
    epochs=mne.make_fixed_length_epochs(data,duration=5,overlap=1)
    array=epochs.get_data()
    return array

In [7]:
sample_data=read_data(a_file_path[1])

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Not setting metadata
198 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 198 events and 2500 original time points ...
0 bad epochs dropped


In [8]:
sample_data.shape #no. of epochs,channels,length of signal

(198, 9, 2500)

In [9]:
%%capture
a_epochs_array=[read_data(i) for i in a_file_path]
f_epochs_array=[read_data(i) for i in f_file_path]

In [10]:
a_epochs_array[1].shape

(198, 9, 2500)

In [11]:
a_epoch_labels=[len(i)*[0] for i in a_epochs_array]
f_epoch_labels=[len(i)*[1] for i in f_epochs_array]
len(a_epoch_labels),len(f_epoch_labels)

(36, 23)

In [12]:
data_list=a_epochs_array+f_epochs_array
label_list=a_epoch_labels+f_epoch_labels

In [13]:
group_list=[[i]*len(j) for i,j in enumerate(data_list)]
len(group_list)

59

In [14]:
group_list[27]

[27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,


In [15]:
data_array=np.vstack(data_list)
label_array=np.hstack(label_list)
group_array=np.hstack(group_list)
print(data_array.shape,label_array.shape,group_array.shape)

(11390, 9, 2500) (11390,) (11390,)


In [16]:
#feature_extraction

In [17]:
np.mean(data_array,axis=-1).shape

(11390, 9)

In [18]:
from scipy import stats

def mean(data):
    return np.mean(data,axis=-1)
    
def std(data):
    return np.std(data,axis=-1)

def ptp(data):
    return np.ptp(data,axis=-1)

def var(data):
        return np.var(data,axis=-1)

def minim(data):
      return np.min(data,axis=-1)


def maxim(data):
      return np.max(data,axis=-1)

def argminim(data):
      return np.argmin(data,axis=-1)


def argmaxim(data):
      return np.argmax(data,axis=-1)

def mean_square(data):
      return np.mean(data**2,axis=-1)

def rms(data): #root mean square
      return  np.sqrt(np.mean(data**2,axis=-1))  

def abs_diffs_signal(data):
    return np.sum(np.abs(np.diff(data,axis=-1)),axis=-1)

def skewness(data):
    return stats.skew(data,axis=-1)

def kurtosis(data):
    return stats.kurtosis(data,axis=-1)

def concatenate_features(data):
    return np.concatenate((mean(data),std(data),ptp(data),var(data),minim(data),maxim(data),argminim(data),argmaxim(data),
                          mean_square(data),rms(data),abs_diffs_signal(data),
                          skewness(data),kurtosis(data)),axis=-1)

In [19]:
from tqdm import tqdm_notebook
features=[]
for data in tqdm_notebook(data_array):
    features.append(concatenate_features(data))
features=np.array(features)
features.shape

C:\Users\91904\AppData\Local\Temp\ipykernel_10692\3783067424.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for data in tqdm_notebook(data_array):


  0%|          | 0/11390 [00:00<?, ?it/s]

(11390, 117)

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
#from numpy import mean
#from numpy import absolute
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold,GridSearchCV,cross_val_score,cross_validate 

In [21]:
#LDA without K fold and Standard Scaler

In [22]:
x_train, x_test, y_train, y_test = train_test_split(features,label_array,test_size=0.2)

model=LinearDiscriminantAnalysis()
model.fit(x_train,y_train)

LinearDiscriminantAnalysis()

In [23]:
# make class predictions for the testing set
y_pred_class = model.predict(x_test)
# calculate accuracy
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

0.7339771729587358


In [24]:
filename = 'models/lda_AF.sav'
joblib.dump(model, filename)

['models/lda_AF.sav']

In [25]:
#LogisticRegression with k-fold and scaler

In [26]:
clf=LogisticRegression()
gkf=GroupKFold(n_splits=5)
param_grid = {'classifier__C': [0.01,0.05,0.1,0.5, 1,2,3,4,5,8, 10,12,15]}
pipe=Pipeline([('scaler',StandardScaler()),('classifier',clf)])
gscv=GridSearchCV(pipe,param_grid,cv=gkf,n_jobs=12)
gscv.fit(features,label_array,groups=group_array)

C:\Users\91904\anaconda3\envs\mne\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=GroupKFold(n_splits=5),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier', LogisticRegression())]),
             n_jobs=12,
             param_grid={'classifier__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4, 5,
                                           8, 10, 12, 15]})

In [27]:
gscv.best_score_

0.5800814383555348

In [28]:
filename = 'models/LR_AF.sav'
joblib.dump(gscv, filename)

['models/LR_AF.sav']

In [29]:
#LDA with k-fold and standard scaler

In [30]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('lda', LinearDiscriminantAnalysis())
])

In [31]:
# Set the number of folds for cross-validation
n_folds = 5

# Create the K-fold cross-validation object
kfold = KFold(n_splits=n_folds,shuffle=True, random_state=42)

# Perform K-fold cross-validation and compute the mean accuracy
scores = cross_val_score(pipeline, x_train, y_train, cv=kfold, scoring='accuracy')
mean_accuracy = np.mean(scores)

In [32]:
# Fit the pipeline on the entire dataset
pipeline.fit(x_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('lda', LinearDiscriminantAnalysis())])

In [33]:
print(mean_accuracy)

0.7406714303692361


In [34]:
y_pred_class = pipeline.predict(x_test)
# calculate accuracy
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

0.7339771729587358


In [35]:
filename = 'models/LDA_Kfold_AF.sav'
joblib.dump(pipeline, filename)

['models/LDA_Kfold_AF.sav']

In [36]:
#XGBoost

In [37]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold

In [38]:
# Create a pipeline with StandardScaler and XGBoost
pipeline1 = Pipeline([
    ('scaler', StandardScaler()),
    ('xgb', xgb.XGBClassifier())
])


In [39]:
# Set the number of folds for cross-validation
n_folds = 5

# Create the K-fold cross-validation object
kfold = KFold(n_splits=n_folds, shuffle=True, random_state=42)

# Perform K-fold cross-validation and compute the mean accuracy
scores = cross_val_score(pipeline1, x_train, y_train, cv=kfold, scoring='accuracy')
mean_accuracy = np.mean(scores)

In [40]:
print(mean_accuracy)

0.9599428692888106


In [41]:
# Fit the pipeline on the entire dataset
pipeline1.fit(x_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('xgb',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=100,
                               n_jobs=None, num_parallel_tree=None,
                               predictor=None, random_state=None, ...))])

In [42]:
y_pred_class = pipeline1.predict(x_test)
# calculate accuracy
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

0.9640035118525022


In [43]:
filename = 'models/XGboost_AF.sav'
joblib.dump(pipeline1, filename)

['models/XGboost_AF.sav']

In [44]:
#Random forest with k-fold and scaler

In [45]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold

In [46]:
# Create a pipeline with StandardScaler and Random Forest
pipeline2 = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier())
])


In [47]:
# Set the number of folds for cross-validation
n_folds = 5

# Create the K-fold cross-validation object
kfold = KFold(n_splits=n_folds, shuffle=True, random_state=42)

# Perform K-fold cross-validation and compute the mean accuracy
scores = cross_val_score(pipeline2, x_train, y_train, cv=kfold, scoring='accuracy')
mean_accuracy = np.mean(scores)


In [48]:
# Fit the pipeline on the entire dataset
pipeline2.fit(x_train, y_train)


Pipeline(steps=[('scaler', StandardScaler()), ('rf', RandomForestClassifier())])

In [49]:
print(mean_accuracy)

0.9302016013218101


In [50]:
y_pred_class = pipeline2.predict(x_test)
# calculate accuracy
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

0.9442493415276558


In [51]:
filename = 'models/RF_Kfold_AF.sav'
joblib.dump(pipeline2, filename)

['models/RF_Kfold_AF.sav']